The goal is to estimate the value of Y column (n x 1) by $\hat{Y}$, where
\begin{equation}
\hat{Y} = \sum_{i=1}^{p} \beta_{i}*X^{i} + \beta_{0}
\end{equation}
where p is the degree of the polynomial.

In [21]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler


#Mean Squared error.
def MSE(B:np.ndarray, X:np.ndarray, y:np.ndarray, already_norm: bool = False) ->float:
    """"The functions computes the mean square error between BX and Y"""
    Predicted = np.matmul(X, B)

    #Normalize the output predicted values if they aren't yet.
    if already_norm == False:
        Predicted_normalized = MinMaxScaler().fit_transform(Predicted)
        y_normalized = MinMaxScaler().fit_transform(y)


    Error: float = np.sum((Predicted_normalized - y_normalized)**2)

    return Error

#Mean absolute error.
def MAE(B:np.ndarray, X:np.ndarray, y:np.ndarray, already_norm: bool = False) -> float:
    """"The functions computes the mean absolute error between BX and Y"""
    Predicted = np.matmul(X, B)

  #Normalize the output predicted values if they aren't yet.
    if already_norm == False:
        Predicted_normalized = MinMaxScaler().fit_transform(Predicted)
        y_normalized = MinMaxScaler().fit_transform(y)


    Error: float = np.sum(np.abs(Predicted_normalized - y_normalized))

    return Error



def R_squared(B:np.ndarray, X:np.ndarray, y:np.ndarray) ->float:
    """"The functions computes the mean absolute error between BX and Y"""

    SS_res = MSE(B, X, y)
    SS_var = np.var(y)

    R = 1 - SS_res/SS_var

    return R




class PolynRegression():
    def __init__(self,Dataset:np.ndarray, tar_var:int, predictor: int ,p:int, train_size, 
                 dx:float = 10**(-8), eps:float =  10**(-8), lr:float =  0.001 ) -> None:
        """Constructor of the Polynomial Regression. p is a fixed degree of the polynomial. Dataset is a dataset the regression is learning from.
        predictor is the id of the predicting feature and tar_var is a predicted feature. Train_size is a fraction of the Dataset used for learning. 0 < train_size <=1"""
        #the dataset we will be learning from.
        self.Dataset = Dataset

        #id of the targer variable
        self.tar_var = tar_var
        #id of the predictor
        self.predictor = predictor
        
        #the degree of the polynomial.
        self.p = p
        #the size of training dataset.
        self.train_size = train_size
        #the size of testing dataset.
        self.test_size = 1 - train_size
        #the offset for derivatives.
        self.dx = dx
        #the division-by-zero preventing value.
        self.eps = eps
        #the learning rate.
        self.lr = lr






0.0
